In [1]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Model, load_model, model_from_json
from keras.layers import Dense, Embedding, Bidirectional, Input, TimeDistributed
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np
import sys
import h5py
import os
import tensorflow as tf
# tf.enable_eager_execution()

from tensorflow.nn.rnn_cell import LSTMStateTuple
from tensorflow.keras.layers import TimeDistributed
from tensorflow.contrib.rnn import OutputProjectionWrapper

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#parameters
maxlen = 30
labels = 2

In [3]:
czech = pd.read_excel('czech.xlsx', encoding='latin',header = None)
czech.columns = ['Name', 'm_or_f']
czech['namelen'] = [len(str(i)) for i in czech['Name']]


In [4]:
cname = czech['Name']
collect = []


for i in range(len(cname)):
    collect.extend(list(str(cname[i]).lower()))
# collect.extend(['END'])
collect = set(collect)

czech.head()

,Name,m_or_f,namelen
0,Abigail,f,7
1,Ada,f,3
2,Adalberta,f,9
3,Adéla,f,5
4,Adelaida,f,8


In [5]:
data_set = pd.read_csv("gender_data.csv",header=None)
data_set.columns = ['name','m_or_f']
data_set['namelen']= [len(str(i)) for i in data_set['name']]
data_set1 = data_set[(data_set['namelen'] >= 2) ]

In [6]:
data_set1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [7]:
names = data_set['name']
gender = data_set['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
vocab = vocab.union(collect)
len_vocab = len(vocab)

In [8]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of data_set is ",len(data_set1))

{'7', 'ú', 'm', 'c', 'd', 'b', 'a', '0', 'g', 'ů', 'ť', 'ř', 'v', 'ó', '3', ' ', 'ý', '6', 'w', '2', 'z', 'e', 'j', 'á', '1', 'p', '4', 'é', 'ě', 'š', 'ž', 'ň', '5', 't', 'x', 'q', 'END', 'č', 'f', 'o', 'i', 'n', 'r', 'u', '8', '9', 'í', 'ď', 'l', 's', 'y', 'h', '.', 'k'}
vocab length is  54
length of data_set is  15226


In [9]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [10]:
print(char_index)

{'7': 0, 'ú': 1, '0': 7, 'ť': 10, 'd': 4, 'a': 6, 'g': 8, 'ř': 11, 'v': 12, 'ó': 13, '3': 14, 'ů': 9, ' ': 15, 'ý': 16, 'm': 2, 'c': 3, 'w': 18, '2': 19, 'z': 20, 'e': 21, 'j': 22, 'b': 5, 'á': 23, '1': 24, 'p': 25, '4': 26, 'é': 27, '6': 17, 'ě': 28, 'ň': 31, 'š': 29, 'ž': 30, '5': 32, 't': 33, 'q': 35, 'END': 36, 'č': 37, 'f': 38, 'o': 39, 'i': 40, 'n': 41, 'r': 42, 'u': 43, '8': 44, 'í': 46, '9': 45, 'l': 48, 's': 49, 'y': 50, 'h': 51, '.': 52, 'x': 34, 'k': 53, 'ď': 47}


In [11]:
#train test split
msk = np.random.rand(len(data_set1)) < 0.9
train = data_set1[msk]
test = data_set1[~msk]     


In [12]:
def set_flag(i):
    tmp = np.zeros(len_vocab);
    tmp[i] = 1
    return(tmp)

In [13]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [14]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    
train_X=np.asarray(train_X)
train_Y=np.asarray(train_Y)

In [15]:
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])
    
test_X = np.asarray(test_X)
test_Y = np.asarray(test_Y)

In [16]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(1538, 30, 54)
(1538, 2)


In [17]:
vtrain_x = []
vtrain_y = []

train_name = [str(i) for i in czech.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtrain_x.append(tmp)
for i in czech.m_or_f:
    if i == 'm':
        vtrain_y.append([1,0])
    else:
        vtrain_y.append([0,1])
vtrain_x = np.asarray(vtrain_x)
vtrain_y = np.asarray(vtrain_y)

#### build model in Tensorflow ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [24]:
''' Parameters '''
tf.reset_default_graph()
no_units=524 

features=train_X.shape[-1] #60
# new_phoneme=new_phoneme.reshape((-1,1,features))

### Making Placeholders

In [25]:
train = tf.placeholder(dtype = tf.float32, shape = [None, train_X.shape[1], train_X.shape[2]])
target = tf.placeholder(dtype = tf.float32, shape = [None, 2])

In [26]:
encoder_fw = [tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]
encoder_bw = [tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]

for i in range(1,3):

    encoder_fw.append(tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]))
    encoder_bw.append(tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(num_units=no_units, initializer=tf.keras.initializers.glorot_normal(), state_is_tuple=True)]))


In [27]:


encoder_outputs, encoder_fw_state, encoder_bw_state = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
                                                                           cells_fw = encoder_fw,
                                                                           cells_bw = encoder_bw,
                                                                           inputs = train,
                                                                           dtype = tf.float32
                                                                          )

In [29]:
# encoder_outputs = tf.concat((encoder_fw_output, encoder_bw_output), 2)

encoder_state_c = tf.concat((encoder_fw_state[-1][0].c, encoder_bw_state[-1][0].c), 1)

encoder_state_h = tf.concat((encoder_fw_state[-1][0].h, encoder_bw_state[-1][0].h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_state_c,
    h=encoder_state_h
)

In [30]:
output=TimeDistributed(tf.layers.Dense(2),input_shape=(encoder_outputs.shape[1],encoder_outputs.shape[2]))(encoder_outputs)

In [32]:
losss=tf.losses.softmax_cross_entropy(output[:,-1,:], target)


In [262]:
losx=[]
saver=tf.train.Saver()

# with tf.device('/gp):
with tf.device('/gpu:0'):
    with tf.Session() as sess:
        
        sess.run(tf.global_variables_initializer())
#         if(tf.train.checkpoint_exists("checkpoint_mfcc_AAI/model.ckpt")):
#             saver.restore(sess, "checkpoint_mfcc_AAI/model.ckpt")
        for i in trange(10):
            loss,_=sess.run([losss,trainop],feed_dict={encoder_input:xtrain})
            losx.append([loss])

        save_path = saver.save(sess, "checkpoint_seq2seq_AAI/model.ckpt")
        print("Model saved in path: %s" % save_path)


100%|██████████| 1000/1000 [19:09<00:00,  1.13s/it]


Model saved in path: checkpoint_seq2seq_AAI/model.ckpt


In [31]:
#build the model: 2 stacked LSTM
print('Build model...')
input_bilstm=Input(shape = (maxlen,len_vocab))
bi_one = Bidirectional(LSTM(512, return_sequences=True))(input_bilstm)
drop1 = Dropout(0.2)(bi_one)
bi_two = Bidirectional(LSTM(512, return_sequences=False))(drop1)
drop2 = Dropout(0.2)(bi_two)
output = Dense(2, activation='softmax')(drop2)
model = Model(input_bilstm, output)


optimizer = optimizers.adam(lr = 0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_acc',patience=10, verbose=1)
model_checkpoint = ModelCheckpoint('Martin_program.hdf5',monitor='val_acc',save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_acc',factor=0.5, patience=5, min_lr=0.0001, verbose=1)
print('Model Built')
print("Weights are being imported from previously trained model..")
# model = load_model('Martin_program.hdf5')
print("Weights imported")

Build model...
Model Built
Weights are being imported from previously trained model..
Weights imported


In [23]:
batch_size=500
loaded_model.fit(train_X, train_Y,
          batch_size=batch_size,
          epochs=10,
#           callbacks=[ model_checkpoint,reduce_lr,early_stopping],
          validation_data=(vtrain_x, vtrain_y),
          verbose = 2
         )
# model.save('model.h5')


Train on 13721 samples, validate on 1250 samples
Epoch 1/10
 - 16s - loss: 0.8588 - acc: 0.5744 - val_loss: 0.7034 - val_acc: 0.5312
Epoch 2/10
 - 15s - loss: 0.5976 - acc: 0.6696 - val_loss: 0.6760 - val_acc: 0.5872
Epoch 3/10


KeyboardInterrupt: 

In [39]:
# json_file = open('model_num.json', 'r')

# loaded_model_json = json_file.read()
# json_file.close()

# loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model.hdf5")

# model.save('model_um.hdf5')
# loaded_model = load_model('Martin_program.hdf5')

In [40]:
score, acc = loaded_model.evaluate(vtrain_x, vtrain_y)
print('Test score:', score)
print('Test accuracy:', acc)

# pred = new_model.predict(vtrain_x)
# r2_score(np.around(pred), vtrain_y)

1250/1250 [==============================] - 1s 853us/step
Test score: 1.118419617319107
Test accuracy: 0.4328


In [29]:
# name=[sys.argv[1]]
name = ["Shivam"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=loaded_model.predict(np.asarray(X))
pred = pred.round()[0]
if pred[0] == 1.0:
    print( name[0], ".... is name of a male")
else:
    print(name[0], ".... is name of a female")

Shivam .... is name of a female
